In [ ]:
import glob
import xarray as xr
import geopandas as gpd
import pandas as pd

In [ ]:
# TIFs downloaded from https://www.eorc.jaxa.jp/ALOS/en/dataset/gmw_e.htm
path = "/Users/alex/Downloads/GMW-15_2020_v3.0/*v3.tif"

xarrays = []
for i in glob.glob(path):
    xarrays.append(xr.open_dataset(i))

In [ ]:
from dea_tools.spatial import xr_vectorize

vectors = []
for ds in xarrays:
    # Rename band_data to data
    ds = ds.rename({"band_data": "data"})

    vector = xr_vectorize(
        ds,
    )
    # Only keep where the attribute is 1.0
    vector = vector.where(vector["attribute"] == 1.0).dropna()
    vectors.append(vector)

In [ ]:
from dep_tools.grids import PACIFIC_EPSG

# Concat all the gdfs into one
gdf = gpd.GeoDataFrame(pd.concat(vectors, ignore_index=True))

# Reproject to pacific
pacific = gdf.to_crs(PACIFIC_EPSG)
# Save to file
pacific.to_parquet("src/gmw_pacific_new.parquet")

In [ ]:
gdf.explore()